In [ ]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = 'mypassword'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

# Supply Chain Optimization

## Description of the use case
This use case involves optimizing delivery routes in the supply chain to reduce costs and improve efficiency. By using geospatial data, businesses can determine the most efficient routes for delivery vehicles.

## Redis Data Structures Used
- **GeoSpatial**: To store and query location data of delivery points and vehicles.
- **Hashes**: To store additional information about delivery points and vehicles.


In [ ]:
# Mock data for the use case
delivery_points = {
    'dp_1': {'name': 'Warehouse A', 'longitude': -73.935242, 'latitude': 40.730610},
    'dp_2': {'name': 'Customer B', 'longitude': -74.0060, 'latitude': 40.7128},
    'dp_3': {'name': 'Customer C', 'longitude': -73.935242, 'latitude': 40.730610},
    'dp_4': {'name': 'Customer D', 'longitude': -73.9911, 'latitude': 40.7305}
}

# Use GeoSpatial to store location data
for dp_id, dp_info in delivery_points.items():
    r.geoadd('delivery_points', (dp_info['longitude'], dp_info['latitude'], dp_id))
    r.hset(dp_id, mapping={'name': dp_info['name'], 'longitude': dp_info['longitude'], 'latitude': dp_info['latitude']})

# Function to find the nearest delivery points
def find_nearest_delivery_points(longitude, latitude, radius_km):
    return r.georadius('delivery_points', longitude, latitude, radius_km, unit='km', withdist=True, sort='ASC')

# Example: Find delivery points within 2 km of a given location
nearest_points = find_nearest_delivery_points(-73.935242, 40.730610, 2)
print('Nearest Delivery Points:')
for point in nearest_points:
    location, distance = point
    print(f"The delivery point {location} is {distance} km away")

# Print the delivery points information
for dp_id in delivery_points.keys():
    print(f"{dp_id}: {r.hgetall(dp_id)}")
